In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=VkmoOWqLEbQYTDPXEVOVKuqA5psGxEFGXYkNG9-ywsg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgKY0gRrWjQDiR_kna4g5LnU-u0UPs8xL1oNUtkPVoQU1pHEAysjfE

Successfully saved authorization token.


In [ ]:

# Algorithm:
# 1. Get huc value for watershed of interest (see table below)
# 3. Create a new feature with selected watershed
# 5. Union the counties (if necessary), export the counties
# 6. Get the CDL data
# 7. Clip to the area of the watershed
# 8. Export 

#--------------------------------#
#   Name           |    huc8     #
#--------------------------------#
#   Lower Boise    |  17050114   #
#--------------------------------#




In [2]:
wshd = ee.FeatureCollection("USGS/WBD/2017/HUC08")

lbrb_wshd = wshd.filterMetadata('name', 'equals', 'Lower Boise')

#print to map
map = emap.Map()
map.addLayer(ee.Image().paint(lbrb_wshd, 0, 2), {}, 'Lower Boise River Basin')
map.addLayerControl()
map

In [30]:
dataset = ee.ImageCollection('USDA/NASS/CDL').filterBounds(lbrb_wshd)
lbrb_cropland = dataset.select('cropland') \
                    .map(lambda image: image.clip(lbrb_wshd))
croplandVis = {
  'min': 1.0,
  'max': 254.0,
  'palette': ['ffd300', #1 corn
              'ff2626', #2 cotton
              '00a832', #3 rice
              'ff9e0a', #4 sorghum
              '267000', #5 soybeans
              'ffff00', #6 sunflower
              '000000', #7 no value
              '000000', #8 no value
              '000000', #9 no value
              '70a500', #10 peanuts
              '00af49', #11 tobacco
              'dda50a', #12 sweet corn
              'dda50a', #13 pop or orn corn
              '7cd3ff', #14 mint
              '000000', #15 no value
              '000000', #16 no value
              '000000', #17 no value
              '000000', #18 no value
              '000000', #19 no value
              '000000', #20 no value
              'e2007c', #21 Barley
              '896054', #22 durum wheat
              'd8b56b', #23 spring wheat
              'a57000', #24 winter wheat
              'd69ebc', #25 other small grains
              '707000', #26 Dbl crop winwht/soybeans
              'aa007c', #27 rye
              'a05989', #28 oats
              '700049', #29 millet
              'd69ebc', #30 speltz
              'd1ff00', #31 canola
              '7c99ff', #32 flaxseed
              'd6d600', #33 safflower
              'd1ff00', #34 rape seed
              '00af49', #35 mustard
              'ffa5e2', #36 alfalfa
              'a5f28c', #37 other hay/non alfalfa
              '00af49', #38 camelina
              'd69ebc', #39 buckwheat
              '000000', #40 no value
              'a800e2', #41sugarbeets
              'a50000', #42 dry beans
              '702600', #43 potatoes
              '00af49', #44 other crops
              'af7cff', #45 sugarcane
              '702600', #46 sweet potato
              'ff6666', #47 misc vegs and fruits
              'ff6666', #48 watermelons
              'ffcc66', #49 onions
              'ff6666', #50 cucumbers
              '00af49', #51 chickpeas
              '00ddaf', #52 lentils
              '54ff00', #53 peas
              'f2a377', #54 tomatoes
              'ff6666', #55 caneberries
              '00af49', #56 hops
              '7cd3ff', #57 herbs
              'e8bfff', #58 clover/wildflowers
              'afffdd', #59 sod
              '00af49', #60 switchgrass
              'bfbf77', #61 fallow/ idle cropland
              '000000', #62 no value
              '93cc93', #63 forest (deciduous, evergreen, mixed)
              'c6d69e', #64 shrubland
              'ccbfa3', #65 barren
              'ff00ff', #66 cherries
              'ff8eaa', #67 peaches, prunes, plums, nectarines, apricots
              'ba004f', #68 apples
              '704489', #69 grapes
              '007777', #70 Christmas trees
              'af9970', #71 other tree crops, pears, pomegranates
              'ffff7c', #72 citrus
              '000000', #73 no value
              'b5705b', #74 pecans
              '00a582', #75 almonds
              'e8d6af', #76 walnuts
              'af9970', #77 pears
              '000000', #78 no value
              '000000', #79 no value
              '000000', #80 no value
              'f2f2f2', #81 clouds/no data
              '999999', #82 developed
              '4970a3', #83 water 
              '000000', #84 no value
              '000000', #85 no value
              '000000', #86 no value
              '7cafaf', #87 wetlands (woody and herbaceous)
              'e8ffbf', #88 nonag/undefined, grassland/pasture
              '000000', #89 no value
              '000000', #90 no value
              '000000', #91 no value
              '00ffff', #92 aquaculture
              '000000', #93 no value
              '000000', #94 no value
              '000000', #95 no value
              '000000', #96 no value
              '000000', #97 no value
              '000000', #98 no value
              '000000', #99 no value
              '000000', #100 no value
              '000000', #101 no value
              '000000', #102 no value
              '000000', #103 no value
              '000000', #104 no value
              '000000', #105 no value
              '000000', #106 no value
              '000000', #107 no value
              '000000', #108 no value
              '000000', #109 no value
              '000000', #110 no value
              '4970a3', #111 open water
              'd3e2f9', #112 perenial ice/snow
              '000000', #113 no value
              '000000', #114 no value
              '000000', #115 no value
              '000000', #116 no value
              '000000', #117 no value
              '000000', #118 no value
              '000000', #119 no value
              '000000', #120 no value
              '999999', #121 developed/open space
              '999999', #122 developed/low intensity
              '999999', #123 developed/med intensity
              '999999', #124 developed/high intensity
              '000000', #125 no value
              '000000', #126 no value
              '000000', #127 no value
              '000000', #128 no value
              '000000', #129 no value
              '000000', #130 no value 
              'ccbfa3', #131 barren
              '000000', #132 no value
              '000000', #133 no value
              '000000', #134 no value
              '000000', #135 no value
              '000000', #136 no value
              '000000', #137 no value 
              '000000', #138 no value
              '000000', #139 no value
              '000000', #140 no value
              '93cc93', #141 deciduous forest
              '93cc93', #142 evergreen forest
              '93cc93', #143 mixed forest
              '000000', #144 no value
              '000000', #145 no value
              '000000', #146 no value
              '000000', #147 no value
              '000000', #148 no value
              '000000', #149 no value 
              '000000', #150 no value
              '000000', #151 no value
              'c6d69e', #152 shrubland
              '000000', #153 no value
              '000000', #154 no value
              '000000', #155 no value
              '000000', #156 no value
              '000000', #157 no value
              '000000', #158 no value 
              '000000', #159 no value
              '000000', #160 no value
              '000000', #161 no value
              '000000', #162 no value
              '000000', #163 no value
              '000000', #164 no value
              '000000', #165 no value
              '000000', #166 no value
              '000000', #167 no value 
              '000000', #168 no value
              '000000', #169 no value
              '000000', #170 no value
              '000000', #171 no value
              '000000', #172 no value 
              '000000', #173 no value
              '000000', #174 no value
              '000000', #175 no value
              'e8ffbf', #176 grassland/pasture
              '000000', #177 no value
              '000000', #178 no value
              '000000', #179 no value
              '000000', #180 no value 
              '000000', #181 no value
              '000000', #182 no value
              '000000', #183 no value
              '000000', #184 no value
              '000000', #185 no value
              '000000', #186 no value
              '000000', #187 no value
              '000000', #188 no value
              '000000', #189 no value 
              '7cafaf', #190 woody wetlands
              '000000', #191 no value
              '000000', #192 no value
              '000000', #193 no value
              '000000', #194 no value
              '7cafaf', #195 herbaceous wetlands
              '000000', #196 no value
              '000000', #197 no value 
              '000000', #198 no value
              '000000', #199 no value
              '000000', #200 no value
              '000000', #201 no value
              '000000', #202 no value
              '000000', #203 no value
              '00ff8c', #204 pistachios
              'd69ebc', #205 triticale
              'ff6666', #206 carrots
              'ff6666', #207 asparagus
              'ff6666', #208 garlic
              'ff6666', #209 cantaloupes
              'ff8eaa', #210 prunes
              '334933', #211 olives
              'e27026', #212 oranges
              'ff6666', #213 honeydew melons
              'ff6666', #214 broccoli
              '739755', #215 avocados
              'ff6666', #216 peppers
              'af9970', #217 pomegrantes
              'ff8eaa', #218 nectarines
              'ff6666', #219 greens
              'ff8eaa', #220 plums
              'ff6666', #221 strawberries
              'ff6666', #222 squash
              'ff8eaa', #223 apricots
              '00af49', #224 vetch
              'ffd300', #225 double crop winwheat/corn
              'ffd300', #226 dbl crop oats/corn
              'ff6666', #227 lettuc
              'f8d248', #228 dbl crop triticale/corn
              'ff6666', #229 pumpkins
              '896054', #230 dbl crop lettuce/durum wheat
              'ff6666', #231 dbl crop lettuce/cantaloupe
              'ff2626', #232 cbl crop lettuce/cotton
              'e2007c', #233 dbl crop lettuce/barley
              'ff9e0a', #234 dbl crop durum wheat/sorghum
              'ff9e0a', #235 dbl crop barley/sorghum
              'a57000', #236 dbl crop winwht/sorghum
              'ffd300', #237 dbl crop barley/corn
              'a57000', #238 dbl crop winwht/cotton
              '267000', #239 dbl crop soybeans/cotton
              '267000', #240 dbl crop soybeans/oats
              'ffd300', #241 dbl crop corn/soybeans
              '000099', #242 blueberries
              'ff6666', #243 cabbage
              'ff6666', #244 cauliflower
              'ff6666', #245 celery
              'ff6666', #246 radishes
              'ff6666', #247 turnips
              'ff6666', #248 eggplants
              'ff6666', #249 gourds
              'ff6666', #250 cranberries
              '000000', #251 no value
              '000000', #252 no value
              '000000', #253 no value
              '267000', #254 dbl crop barley/soybeans
  ]}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(lbrb_cropland, croplandVis, 'Cropland')
Map


In [ ]:
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'datasets'}  # scale means resolution.
img_lst = srb_landcover.toList(100)

count = srb_landcover.size().getInfo()

for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    name = image.get('system:index').getInfo()
    print(name)
    task = ee.batch.Export.image(image.clip(srb_bnd.geometry()), name, downConfig)
    task.start()
    print(task.status())

NLCD1992
{'state': 'READY', 'description': 'NLCD1992', 'creation_timestamp_ms': 1602450996161, 'update_timestamp_ms': 1602450996161, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'D35E3TPVLPHPIXST7VZDLMRD', 'name': 'projects/earthengine-legacy/operations/D35E3TPVLPHPIXST7VZDLMRD'}
NLCD2001
{'state': 'READY', 'description': 'NLCD2001', 'creation_timestamp_ms': 1602451018960, 'update_timestamp_ms': 1602451018960, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UNT7DBP4SQRBURSUVD466POV', 'name': 'projects/earthengine-legacy/operations/UNT7DBP4SQRBURSUVD466POV'}
NLCD2004
{'state': 'READY', 'description': 'NLCD2004', 'creation_timestamp_ms': 1602451040720, 'update_timestamp_ms': 1602451040720, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'MXGCVL6NHIIBKYX62RCAY43D', 'name': 'projects/earthengine-legacy/operations/MXGCVL6NHIIBKYX62RCAY43D'}
NLCD2006
{'state': 'READY', 'description': 'NLCD2006', 'creation_timestamp_ms': 1602451059662, 'update_timestamp

In [ ]:
task.status()

{'creation_timestamp_ms': 1602451143507,
 'description': 'NLCD2016',
 'id': 'VHMYVOWJI5VC7UR4CGUYWXAS',
 'name': 'projects/earthengine-legacy/operations/VHMYVOWJI5VC7UR4CGUYWXAS',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1602451143507}